In [1]:
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver # 브라우저를 조작하는 모듈
from selenium.webdriver.chrome.service import Service # 크롬 드라이버의 시작과 중지를 담당하는 클래스
from selenium.webdriver.common.by import By # 엘리먼트를 어떤 방식으로 선택할 지에 대한 상수
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 관리
import pandas as pd
from tqdm.auto import tqdm
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

#### 저는 명시적 대기를 사용하였지만 암묵적 대기를 사용하시면 더 빠르게 실행되게 할 수도 있습니다.

In [2]:
DATA_PATH1 = "C:/finalproject/"

In [3]:
influencer_list = pd.read_csv(f"{DATA_PATH1}influencer_list.csv", encoding="cp949")
influencer_list

,NO.,ID,인스타 주소,팔로워 수,분야
0,1,ve.rris,https://www.instagram.com/ve.rris/,5.6만,경제
1,2,alzzahwang,https://www.instagram.com/alzzahwang/,1.4만,경제
2,3,vengguri_toon,https://www.instagram.com/vengguri_toon/,11.9만,경제
3,4,hibi_heebee,https://www.instagram.com/hibi_heebee/,6.4만,경제
4,5,donizoa_,https://www.instagram.com/donizoa_/,2.5만,경제
...,...,...,...,...,...
350,351,lovely_szi,https://www.instagram.com/lovely_szi/,11.9만,운동
351,352,jungyun_diet,https://www.instagram.com/jungyun_diet/,4.4만,운동
352,353,o.o.g.i,https://www.instagram.com/o.o.g.i/,2.5만,일상
353,354,kinda__sweet,https://www.instagram.com/kinda__sweet/,4.4만,일상


In [24]:
influencer_list['ID'][0]

've.rris'

In [4]:
address_list = influencer_list["인스타 주소"].iloc[280:]

In [5]:
address_list

280           https://www.instagram.com/hoonjarro
281      https://www.instagram.com/vovo_home_cook
282        https://www.instagram.com/jeongeun0802
283     https://www.instagram.com/haebromom_hanoi
284    https://www.instagram.com/kyungsun_kitchen
                          ...                    
350         https://www.instagram.com/lovely_szi/
351       https://www.instagram.com/jungyun_diet/
352            https://www.instagram.com/o.o.g.i/
353       https://www.instagram.com/kinda__sweet/
354           https://www.instagram.com/jin_0310/
Name: 인스타 주소, Length: 75, dtype: object

In [9]:
DATA_PATH = "C:/finalproject/"

In [10]:
def influence_searching(ID):
    url = "https://www.instagram.com/" + str(ID)
    return url

In [11]:
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, "div._aagw")[0]
    first.click()
    time.sleep(1.5)

In [14]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
    except:
        content = ''
    # 해시 태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    # 작성 일자
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    # 좋아요
    try:
        like = soup.select('section._ae5m._ae5n._ae5o > div > div > span > a > span')[0].text
        like = re.findall(r'[0-9]+', like)[0]
    except:
        like = 0
    
    # 댓글 더보기 클릭
    action = ActionChains(driver)
    for _ in range(100):
        try:
            next_coment = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button')
            action.move_to_element(next_coment).perform()
            driver.implicitly_wait(10)
            next_coment.click()
            driver.implicitly_wait(10)
        except:
            break
    # 아이디 및 댓글 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    try:
        username = [username.div.div.div.a.get_text() for username in soup.find_all('h3', {'class' :'_a9zc'})]
        coments = [coment.get_text() for coment in soup.find_all('span', {'class':'_aacl _aaco _aacu _aacx _aad7 _aade'})]
    except:
        username = ''
        coments = ''

    content = re.sub(r'#[^\s#,\\]+', "", content)
    data = [content, username, coments, date, like, tags]
    return data

In [15]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh')
    right.click()
    time.sleep(3)

In [18]:
# 인스타그램에 로그인
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
# options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get('https://www.instagram.com')
time.sleep(3)

email = str(input('아이디:')) # 아이디 입력
input_id = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(1) > div > label > input').send_keys(email)

password = str(input('비밀번호:')) # 페스워드 입력
input_pw = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(2) > div > label > input').send_keys(password)
time.sleep(1)

driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(3)").click()

time.sleep(5)

In [34]:
# 한명의 인플루언서마다 게시글 100개 크롤링
target = 100
for infuluence in tqdm(address_list, desc="outer", position=0):
    driver.get(infuluence)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span._ac2a")))
    count = int(driver.find_element(By.CSS_SELECTOR, "span._ac2a").text)
    time.sleep(2)
    select_first(driver)
    
    time.sleep(0.5)
    results = []
    for j in tqdm(range(target), desc="inner", position=1):
        if j >= count:
            break
        try:
            data = get_content(driver)
            results.append(data)
            move_next(driver)
        except Exception as e:
            time.sleep(2)
            move_next(driver)

        time.sleep(3)
    infuluence = infuluence.replace("/","")
    infuluence = infuluence.split('.com')[1]
    results_df = pd.DataFrame(results)
    results_df.columns = ['content','username', 'coment', 'date', 'like', 'tags']
    results_df.to_csv(f'{DATA_PATH}_{infuluence}_insta_crawling.csv', encoding='utf-8-sig')
    # 다음 ID로 넘어가기전 너무 빠르게 하면 오류 및 사람인지 확인하는 메시지가 떠서 저는 우선 30초를 줬습니다. 
    # 본인이 해보시고 오류가 안나는 선에서 시간을 정하시면 됩니다.
    
    time.sleep(7)
    
driver.quit()

outer:   0%|          | 0/75 [00:00<?, ?it/s]

inner:   0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
len(influencer_list['ID'][280:])

75

In [1]:
tmp = "https://www.instagram.com/ve.rris/"	
tmp.strip("/")

'https://www.instagram.com/ve.rris'

In [2]:
tmp = tmp.replace("/","")
tmp = tmp.split('.com')[1]
tmp

've.rris'